In [1]:
import random
rand_edge = lambda p: random.random() < p

import pandas as pd
import numpy as np

In [2]:
def write_file(fname, content):
    f = open(fname, "w+")

    f.write(content)
    f.close()

#### Create Node

In [3]:
fname = "data/names.csv"
df = pd.read_csv(fname)

# df.sample(frac=0.1)
df_sample = df.sample(n=100, replace=False)
names = df_sample.name.values.tolist()

print(len(set(names)))
names = list(set(names))
cql = "CREATE \n" + ",".join("(%s : Person {name: \"%s\"})\n"%(name, name) for name in names)

100


#### Partition

In [4]:
# initial partition
i1, i2, i3 = 25, 40, 60

c1 = names[:i1]
c2 = names[i1:i2]
c3 = names[i2:i3]
c4 = names[i3:]
cs = [c1, c2, c3, c4]

print(len(c1), len(c2), len(c3), len(c4))

25 15 20 40


#### Iterate

In [5]:
volatility = 20
cohesity = 0.05

In [6]:
# randomly swap nodes among partitions
for i in range(volatility):
    ia, ib = np.random.choice(range(len(cs)), 2)
    if ia != ib:
        ca, cb = cs[ia], cs[ib]
        cai = np.random.choice(range(len(ca)))
        cbi = np.random.choice(range(len(cb)))
        ca[cai], cb[cbi] = cb[cbi], ca[cai]

# randomly establish edges
rel = """"""
edges = []

for c in cs:
    for src in c:
        for dst in c:
            if src != dst and rand_edge(cohesity):
                rel += ",(%s)-[:KNOWS]->(%s)\n"%(src, dst)
                edges.append((names.index(src), names.index(dst)))

fname = "data/community_evolution/init.cql"
write_file(fname, cql + rel)

# needed for degree centrality
pd.DataFrame({"id": range(len(names)), "name": names}).to_csv("data/community_evolution/init_v.csv", index=False)
pd.DataFrame(edges, columns=["src", "dst"]).to_csv("data/community_evolution/init_e.csv", index=False)

In [7]:
from py2neo import Graph

ip = "192.168.0.101"
graph = Graph("bolt://%s:7687"%ip, password="123456")

graph.run("MATCH (n) DETACH DELETE n;")
graph.run(cql + rel)